## 요약


1. 전처리 함수 : preprocessing(df, col)
2. 크롤링 이슈 때문에 컬럼(회사소개, 주요업무, 자격요건, 우대사항, 혜택 및 복지) 별로 특잇값에 대한 처리해줬음 → 문자열 길이 정상화
3. 기술스택 -> 문장으로 변경 후, combined(주요업무, 자격요건, 우대사항, 기술스택)에 포함시킴
4. 원활한 문장 토큰화를 위해 줄바꿈 문자는 제거하지 않음. -> 문장 토큰화의 기준으로 사용할 예정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, re
import pandas as pd
from glob import glob
from collections import Counter
import random

In [ ]:
wanted = pd.read_csv('/content/drive/MyDrive/응용통계학과 공모전/data/wanted_final.csv')
wanted.head()

,공고명,직무,업종,유사 직무,회사명,회사소개,주요업무,자격요건,우대사항,혜택 및 복지,마감기한,기술스택
0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"""IT, 컨텐츠""","""Data Engineer"",""데이터 엔지니어"",""Machine Learning E...",플리토,"""플리토는 유수의 글로벌 IT 기업들과 함께 '언어 장벽 없는 세계'를 만들어 가고...",- 기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니...,- AI / ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분\n - M...,사항\n- 모델을 직접 만들었거나 공개된 모델을 fine-tuning해서 성능을 향...,"- 상호 존중하는 문화\n : 직급 없는 \""님\"" 호칭 및 경어 사용, 자율 ...","""2023-11-27T00:00:00""","Github, Android, Firebase, Kotlin, Coroutine"
1,AI 컴파일러 개발자 (1년 이상),머신러닝 엔지니어,"""IT, 컨텐츠""","""Machine Learning Engineer"",""머신러닝 엔지니어""",티에스엔랩,"""실시간 AI/실시간 통신 전문 기업 TSN Lab에서 네트워크 AI 개발자를 모십...",[어떤 팀에서 일 하나요?]\n• AI팀에서 일 합니다. AI팀은 AI 모델 만들고...,[어떤 사람을 채용하나요?]\n• 컴파일러 관련 지식이 있어야 합니다.\n• LLV...,NaN,[연봉은 어떻게 되나요?]\n• 경력자의 경우 기존의 경력과 동일한 분야인 경우 기...,NaN,None
2,LLM 연구원 (자연어 처리),머신러닝 엔지니어,"""IT, 컨텐츠""","""Machine Learning Engineer"",""머신러닝 엔지니어""",딥노이드(Deepnoid),"""LLM 연구원 (자연어 처리)\n\n\n",LLM 모델 연구 및 고도화\n\nLLM 연구의 서비스화를 위한 개발 지원\n,"경력 3년 이상 또는 관련 학과 석사 학위 소지자\n\nPyTorch, Tensor...","사항\n인공지능 관련 대회(해커톤, 캐글, 그랜드챌린지 등)의 출전 경험 및 입상\...",1. 유연 근무제\n2. 자유 복장\n3. 최신 장비\n4. 수평적 조직\n5. 도...,"""2024-02-29T00:00:00""","C#, C, C++, WPF"
3,AI R\u0026D 팀장,머신러닝 엔지니어,"""IT, 컨텐츠""","""Machine Learning Engineer"",""머신러닝 엔지니어""",딥노이드(Deepnoid),"""AI R\u0026D 팀장\n\n\n",sLLM 모델 연구개발 및 서비스 고도화 \n\nsLLM 기반의 서비스 모델 개발 ...,AI 관련학과 박사학위 소지자 또는 이에 준하는 경력 보유자\n\nLLM 관련 최신...,"사항\nAI 관련 Top tier 학회(NeurIPS, CVPR, AAAI 등) 논...",1. 유연 근무제\n2. 자유 복장\n3. 최신 장비\n4. 수평적 조직\n5. 도...,"""2024-02-29T00:00:00""","C#, C, C++, WPF"
4,소프트웨어 엔지니어,머신러닝 엔지니어,"""IT, 컨텐츠""","""Web Developer"",""웹 개발자"",""Machine Learning Engi...","스타인펠드(Steinfeld,Inc.)","""Steinfeld.co는 치과 보철치료에 필요한 보철물을 더 효율적으로 제공하기 ...",• AI 개발\n보철물 자동 디자인을 위한 AI 모델 개발\n보철물 자동 디자인 서...,• python 또는 TypeScript 로 개발 경험 있으신분\n• Open mi...,사항\n• 성장하는 스타트업에 관심 있으신 분\n,• 경쟁력 있는 연봉\n - 협의를 통해 충분히 경쟁력 있는 수준의 연봉을 제...,NaN,"Git, Firebase, Google Cloud Platform, Pytorch,..."


In [ ]:
wanted[['공고명','직무','주요업무','자격요건','우대사항','기술스택']]

,공고명,직무,주요업무,자격요건,우대사항,기술스택
0,"AI, ML Engineer 박사급",머신러닝 엔지니어,- 기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니...,- AI / ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분\n - M...,사항\n- 모델을 직접 만들었거나 공개된 모델을 fine-tuning해서 성능을 향...,"Github, Android, Firebase, Kotlin, Coroutine"
1,AI 컴파일러 개발자 (1년 이상),머신러닝 엔지니어,[어떤 팀에서 일 하나요?]\n• AI팀에서 일 합니다. AI팀은 AI 모델 만들고...,[어떤 사람을 채용하나요?]\n• 컴파일러 관련 지식이 있어야 합니다.\n• LLV...,NaN,None
2,LLM 연구원 (자연어 처리),머신러닝 엔지니어,LLM 모델 연구 및 고도화\n\nLLM 연구의 서비스화를 위한 개발 지원\n,"경력 3년 이상 또는 관련 학과 석사 학위 소지자\n\nPyTorch, Tensor...","사항\n인공지능 관련 대회(해커톤, 캐글, 그랜드챌린지 등)의 출전 경험 및 입상\...","C#, C, C++, WPF"
3,AI R\u0026D 팀장,머신러닝 엔지니어,sLLM 모델 연구개발 및 서비스 고도화 \n\nsLLM 기반의 서비스 모델 개발 ...,AI 관련학과 박사학위 소지자 또는 이에 준하는 경력 보유자\n\nLLM 관련 최신...,"사항\nAI 관련 Top tier 학회(NeurIPS, CVPR, AAAI 등) 논...","C#, C, C++, WPF"
4,소프트웨어 엔지니어,머신러닝 엔지니어,• AI 개발\n보철물 자동 디자인을 위한 AI 모델 개발\n보철물 자동 디자인 서...,• python 또는 TypeScript 로 개발 경험 있으신분\n• Open mi...,사항\n• 성장하는 스타트업에 관심 있으신 분\n,"Git, Firebase, Google Cloud Platform, Pytorch,..."
...,...,...,...,...,...,...
1143,DBA,DBA,"• DB 분석/설계/구축/운영 및 관리\n• DB 성능분석, 최적화(Tuning) ...","• Mysql, MariaDB, Aurora 중 1개 이상의 관리 및 운영\n","사항\n• AWS 클라우드 플랫폼에 대한 DB 운영\n• AWS RDS, Auror...","React, HTML, JavaScript"
1144,"데이터베이스 복제 솔루션 개발(C/C++, Linux/Unix)",DBA,• 데이터베이스 변경 로그 추출\n• 데이터 저장 및 추출\n• 데이터베이스\n,• 신입 or 경력\n• C / C++\n,사항\n• 데이터베이스에 대한 전반적인 이해도가 높으신 분\n• 데이터베이스 기반 ...,"Git, Hadoop, Linux, MySQL, Oracle, C / C++, Py..."
1145,[핀트] DBA,DBA,• On-Premise 기반 환경의 데이터베이스를 구축하고 운영합니다.\n• 클라우...,"• 3년 이상의 DBA 경험이(MySQL, PostgreSQL, Maria DB) ...","사항\n• 증권, 카드, 뱅킹 관련 핀테크 산업군의 경험이 있는 분\n• 자동화 작...","Hadoop, Spark, SQL, Data Analysis, Tableau, Am..."
1146,DBA,DBA,핀다 DBA는 아키텍처를 구성하고 Database를 안정적으로 운영합니다.\n• 데...,"• MYSQL DB 운영 경험이 3년 이상 있는 분\n• DB 보안, 백업에 대한 ...","사항\n• 서비스 DB 설계 및 구축 경험 있는 분\n• 플랫폼 DB 운영, 아카이...",None


In [ ]:
df = wanted.copy()

In [ ]:
df.shape

(1148, 12)

In [ ]:
df.nunique()

공고명        653
직무           6
업종          15
유사 직무      212
회사명        451
회사소개       626
주요업무       730
자격요건       740
우대사항       720
혜택 및 복지    762
마감기한        25
기술스택       316
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1148 entries, 0 to 1147
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   공고명      1148 non-null   object
 1   직무       1148 non-null   object
 2   업종       1148 non-null   object
 3   유사 직무    1148 non-null   object
 4   회사명      1148 non-null   object
 5   회사소개     1121 non-null   object
 6   주요업무     1148 non-null   object
 7   자격요건     1148 non-null   object
 8   우대사항     1131 non-null   object
 9   혜택 및 복지  1148 non-null   object
 10  마감기한     169 non-null    object
 11  기술스택     1148 non-null   object
dtypes: object(12)
memory usage: 107.8+ KB


In [ ]:
df.columns

Index(['공고명', '직무', '업종', '유사 직무', '회사명', '회사소개', '주요업무', '자격요건', '우대사항',
       '혜택 및 복지', '마감기한', '기술스택'],
      dtype='object')

In [ ]:
df.isna().sum()

공고명          0
직무           0
업종           0
유사 직무        0
회사명          0
회사소개        27
주요업무         0
자격요건         0
우대사항        17
혜택 및 복지      0
마감기한       979
기술스택         0
dtype: int64

In [ ]:
# 컬럼별 문자열 길이 확인
# 분포가 이상한 컬럼이 있음 -> 회사소개, 주요업무, 자격요건, 우대사항, 혜택 및 복지

num_df = pd.DataFrame()

for col in ['공고명', '직무', '업종', '유사 직무', '회사명', '회사소개', '주요업무', '자격요건', '우대사항', '혜택 및 복지', '마감기한']:
    num_df[f'{col}_num'] = (df[col].dropna().map(lambda x: len(x)))

num_df.describe()

,공고명_num,직무_num,업종_num,유사 직무_num,회사명_num,회사소개_num,주요업무_num,자격요건_num,우대사항_num,혜택 및 복지_num,마감기한_num
count,1148.000000,1148.000000,1148.000000,1148.000000,1148.000000,1121.000000,1148.000000,1148.000000,1131.000000,1148.000000,169.0
mean,20.412892,8.448606,8.607143,79.790070,6.974739,1021.841213,1031.126307,1882.953833,1176.416446,384586.447735,21.0
std,10.636102,1.638137,1.471562,27.656039,4.971257,6901.176746,10254.841119,13655.246743,16089.790688,3184.102485,0.0
min,3.000000,3.000000,4.000000,23.000000,2.000000,15.000000,13.000000,13.000000,13.000000,383891.000000,21.0
25%,12.000000,8.000000,9.000000,58.000000,4.000000,312.000000,95.500000,116.000000,103.000000,384082.000000,21.0
50%,19.000000,9.000000,9.000000,88.000000,6.000000,508.000000,164.000000,174.500000,162.000000,384254.000000,21.0
75%,27.000000,9.000000,9.000000,98.000000,9.000000,854.000000,285.750000,251.000000,253.000000,384564.000000,21.0
max,56.000000,10.000000,13.000000,154.000000,46.000000,116899.000000,168847.000000,113725.000000,271103.000000,440149.000000,21.0


In [ ]:
# 전처리 함수

# html 태그 제거
def remove_tag(content):
    cleaner = re.compile('<.*?>|\[[^)]*\]')
    cleantext = re.sub(cleaner,'',content)
    return cleantext

# 특수문자 제거
def cleanText(readData):
    pattern = re.compile(r'\d+[\.\)]{1}')
    readData = re.sub(pattern, '', readData)
    #text = re.sub('[-=+,#\?:^$.@*\"※~&%ㆍ!『』\\‘|\(\)\[\]\<\>`\'…》\’\“\”\·\n\r\t●■◇◆▶•;◎【】⦁\\\><_＜＞▷①②③④⑤⑥]', '', readData).strip()

    # \\n 제거 x
    text = re.sub('[-=+#\?:^$.@*\"※~&%ㆍ!『』‘|\(\)\[\]\<\>`\'…》\’\“\”\·\r\t●■◇◆▶•;◎【】⦁\><_＜＞▷①②③④⑤⑥✔]', '', readData).strip()
    return text

# http ~ 로 시작하는 문자열, 한글 등장 전까지 삭제
def remove_http_to_korean(text):
    pattern = re.compile(r'http[^\uAC00-\uD7A3]*')
    result = re.sub(pattern, '', text)
    return result

def preprocessing(df, col):
    df[col] = df[col].str.replace(r"\\u0026", "&", regex=True)
    df[col] = df[col].str.replace(r"\u200b", "", regex=True)
    df[col] = df[col].str.replace(r"/", ",", regex=True).str.replace(r",", ", ", regex=True)
    df[col] = df[col].str.strip('"')
    #df[col] = df[col].str.replace(r'\\n',' ', regex=True).str.replace(r'  ',' ').str.strip()

    # 값이 null 이 아닐 때 적용
    df.loc[df[col].isna() == False, col] = df[col].dropna().map(lambda x:remove_tag(x))
    df.loc[df[col].isna() == False, col] = df[col].dropna().map(lambda x:cleanText(x))
    df.loc[df[col].isna() == False, col] = df[col].dropna().map(lambda x:remove_http_to_korean(x))

## 공고명

In [ ]:
df['공고명'].nunique()

653

In [ ]:
# "\\u0026" -> &로 변경
df['공고명'] = df['공고명'].str.replace(r"\\u0026", "&", regex=True)

In [ ]:
df['공고명'].tolist()

['AI, ML Engineer 박사급',
 'AI 컴파일러 개발자 (1년 이상)',
 'LLM 연구원 (자연어 처리)',
 'AI R&D 팀장',
 '소프트웨어 엔지니어',
 'Software Engineer (Full Stack)',
 'AI Engineer (NLP)',
 'Data Scientist',
 'AI 엔지니어',
 '머신비전 엔지니어',
 'AI 엔지니어(주니어)',
 '음성AI Researcher',
 'AI Scientist Director',
 'Machine Learning Engineer (ITS팀)',
 'LLM AI 연구개발자',
 'AI 엔지니어',
 'AI 엔지니어 - Vision',
 'Python application & Algorithm 개발자',
 'Computer Vision AI Engineer',
 '기업분석 데이터 분석가',
 '데이터 사이언티스트',
 '자연어 처리 - AI Engineer (LLM) (1년이상)',
 '자연어 처리 - AI Engineer (LLM) (신입)',
 '[Enterprise] NLP Engineer',
 '머신러닝 엔지니어',
 '딥러닝 엔지니어',
 '[플레이오] 데이터 엔지니어',
 '[신입] 인공지능 신약개발 연구원 (박사)',
 'Senior Software Engineer, Vision ML',
 '웹 프로그래머(2년 이상)',
 'OCR Engineer',
 '[플레이오] 데이터 과학자',
 '모바일 로봇 개발 (SLAM/NAVIGATION)',
 '백엔드 개발자',
 '자연어처리 엔지니어 (NLP)',
 '컴퓨터비전 엔지니어 (CV)',
 '[헥랩 AI랩] AI 개발자 (일반개발 10년 이상)',
 '[500억↑투자] 딥러닝/머신러닝(AI) 개발자 (신입/병역특례 전문연구요원 가능)',
 '[신입] AI Research Engineer(전문연구요원 가능)',
 '신입/주니어 개발자',
 '머신러닝 엔지니어 (ML)',
 'Vision AI Application Engi

## 직무

In [ ]:
df.직무.value_counts()

머신러닝 엔지니어     359
데이터 엔지니어      329
데이터 사이언티스트    206
빅데이터 엔지니어     155
DBA            78
BI 엔지니어        21
Name: 직무, dtype: int64

## 유사직무
- 형태 :  "Machine Learning Engineer", "머신러닝 엔지니어"
- 큰 따옴표 제거
- 매핑되는 영어 직무명 제거하고 한글 직무명만 남겨둠

In [ ]:
df['유사 직무'].head()

0    "Data Engineer","데이터 엔지니어","Machine Learning E...
1              "Machine Learning Engineer","머신러닝 엔지니어"
2              "Machine Learning Engineer","머신러닝 엔지니어"
3              "Machine Learning Engineer","머신러닝 엔지니어"
4    "Web Developer","웹 개발자","Machine Learning Engi...
Name: 유사 직무, dtype: object

In [ ]:
df['유사 직무'] = df['유사 직무'].str.replace('"', '')

In [ ]:
df['유사 직무'].head()

0    Data Engineer,데이터 엔지니어,Machine Learning Engine...
1                  Machine Learning Engineer,머신러닝 엔지니어
2                  Machine Learning Engineer,머신러닝 엔지니어
3                  Machine Learning Engineer,머신러닝 엔지니어
4    Web Developer,웹 개발자,Machine Learning Engineer,...
Name: 유사 직무, dtype: object

In [ ]:
# 쉼표로 구분된 값을 리스트로 변환하고 모든 직무를 담은 리스트 생성
all_jobs = [job for jobs in df['유사 직무'].apply(lambda x: x.split(',')) for job in jobs]

# 리스트를 set으로 변환하여 중복을 제거하고 유니크한 직무 확인
unique_jobs = set(all_jobs)

In [ ]:
unique_jobs
# BI 엔지니어, C++ 개발자, DevOps / 시스템 관리자, iOS 개발자, VR 엔지니어, PHP 개발자, Node.js 개발자, ERP전문가

{' Network Administrator',
 ' Test Engineer',
 ' Voice Engineer',
 'Android Developer',
 'BI Engineer',
 'BI 엔지니어',
 'Back-end Engineer',
 'Big Data Engineer',
 'C',
 'C++ Developer',
 'C++ 개발자',
 'CIO',
 'CTO',
 'Chief Information Officer',
 'Chief Technology Officer',
 'Cross-platform App Developer',
 'DBA',
 'Data Engineer',
 'Data Scientist',
 'Database Administrator',
 'DevOps / System Admin',
 'DevOps / 시스템 관리자',
 'Development Manager',
 'ERP Consultant',
 'ERP전문가',
 'Embedded Developer',
 'Front-end Engineer',
 'Graphics Engineer',
 'Hardware Engineer',
 'Java Developer',
 'Machine Learning Engineer',
 'Node.js Developer',
 'Node.js 개발자',
 'PHP Developer',
 'PHP 개발자',
 'Product Manager',
 'Python Developer',
 'QA',
 'Security Engineer',
 'Software Engineer',
 'System',
 'Technical Support',
 'VR Engineer',
 'VR 엔지니어',
 'Video',
 'Web Developer',
 'iOS Developer',
 'iOS 개발자',
 '개발 매니저',
 '그래픽스 엔지니어',
 '기술지원',
 '네트워크 관리자',
 '데이터 사이언티스트',
 '데이터 엔지니어',
 '머신러닝 엔지니어',
 '보안 엔지니어',
 '빅데

In [ ]:
# 예외 목록 정의
exception_list = ['BI 엔지니어', 'C++ 개발자', 'DevOps / 시스템 관리자', 'iOS 개발자', 'VR 엔지니어', 'PHP 개발자', 'Node.js 개발자', 'ERP전문가']

# 영어가 하나라도 포함된 직무를 지우는 함수
def remove_english_jobs(job_list):
    result = []
    for job in job_list:
        if any(char.isalpha() and char.isascii() for char in job) and job not in exception_list:
            continue
        result.append(job)
    return result

# '유사 직무' 컬럼에 적용
df['유사 직무'] = df['유사 직무'].apply(lambda x: ','.join(remove_english_jobs(x.split(','))))

df['유사 직무'].head(10)

0               데이터 엔지니어,머신러닝 엔지니어
1                        머신러닝 엔지니어
2                        머신러닝 엔지니어
3                        머신러닝 엔지니어
4       웹 개발자,머신러닝 엔지니어,소프트웨어 엔지니어
5      서버 개발자,머신러닝 엔지니어,소프트웨어 엔지니어
6     파이썬 개발자,데이터 사이언티스트,머신러닝 엔지니어
7    데이터 엔지니어,데이터 사이언티스트,머신러닝 엔지니어
8     영상,음성 엔지니어,C++ 개발자,머신러닝 엔지니어
9     C++ 개발자,머신러닝 엔지니어,소프트웨어 엔지니어
Name: 유사 직무, dtype: object

## 업종

In [ ]:
df.업종.head()

0    "IT, 컨텐츠"
1    "IT, 컨텐츠"
2    "IT, 컨텐츠"
3    "IT, 컨텐츠"
4    "IT, 컨텐츠"
Name: 업종, dtype: object

In [ ]:
df['업종'] = df['업종'].str.strip('"')

In [ ]:
df['업종'].unique()

array(['IT, 컨텐츠', '기타 서비스업', '전문, 과학기술', '제조', '보건, 사회복지', '교육', '농림어업',
       '금융', '판매, 유통', '물류, 운송', '게임', '예술, 스포츠, 여가', '부동산', '사업지원',
       '숙박, 음식점'], dtype=object)

In [ ]:
df['업종'].nunique()

15

## 회사명

In [ ]:
df['회사명'].nunique()

451

In [ ]:
df['회사명'].value_counts()

넥슨코리아(NEXON)       20
하나금융티아이            14
씨제이대한통운(CJ대한통운)    14
포티투닷(42dot)        12
휴톰                 11
                   ..
쉐어트리츠               1
메타넷글로벌              1
로이드케이               1
더리얼그룹               1
아크데이타               1
Name: 회사명, Length: 451, dtype: int64

## 회사소개

In [ ]:
df['회사소개'].nunique()

626

In [ ]:
preprocessing(df, '회사소개')

In [ ]:
df['회사소개'] = df['회사소개'].str.split('user').str[0]
df['회사소개'] = df['회사소개'].str.split('companyname').str[0]

In [ ]:
df['회사소개'].tolist()

['플리토는 유수의 글로벌 IT 기업들과 함께 언어 장벽 없는 세계를 만들어 가고 있는 언어 플랫폼 기업입니다\\n\\n\\n자체 엔진을 적용한 플리토 AI번역과 1, 000만 명 이상의 글로벌 유저가 직접 참여하는 플리토 집단지성 번역,  그리고 언어 능력을 활용해 즐기며 리워드를 모을 수 있는 플리토 아케이드Arcade 서비스를 제공하고 있으며,  이 외에도 언어 장벽이 없는 세상를 위한 가치있는 신규 서비스들을 끊임없이 개발하고 있습니다 \\n\\n\\n이처럼 다양한 서비스를 통해 확보한 가장 자연스럽고 정확한 언어 데이터를 글로벌 IT 기업에 AI 학습 데이터로 제공하고 있으며,  향후 수십조 규모로 성장할 언어 데이터 시장에서 한국 대표 기업으로서의 입지를 굳혀가고 있습니다\\n\\n\\n언어의 장벽을 뛰어넘는 독보적 언어 데이터 전문 기업 플리토에서 \\n함께 성장하기 위한 즐거운 도전에 합류하실 많은 분들의 지원을 기다립니다\\n\\n\\n\\n  \\n  근무지  플리토 서울 본사 2호선 삼성역 도보 3분 거리 위치\\n  근무시간  10시19시 기본 금요일은 플렉서블 근무 811시 출근 선택\\n  근로계약  정규직 회사 내규에 의해 3개월의 수습기간 적용\\n\\n\\n \\n  입사지원  20231127월 1159까지\\n  전형진행  서류전형  테스트전형  면접전형  입사협의  입사\\n    빠른 채용 진행을 위해 면접전형시 경력, 소득 증빙자료를 요청합니다\\n    전형 결과에 상관없이 결과 확정시 개별 메일로 안내드립니다\\n\\n\\n \\n  국가보훈자 및 국가등록장애인은 관련 법에 의거하여 우대합니다\\n  제출하신 지원서 내용이 사실과 다르거나 허위 서류를 제출한 경우 채용취소 등 불이익을 받을 수 있습니다\\n\\n\\n',
 '실시간 AI, 실시간 통신 전문 기업 TSN Lab에서 네트워크 AI 개발자를 모십니다\\n\\n\\n TSN Lab은 엄청난 속도로 성장하는 실시간 AI 기술과 실시간 네트워크 회사입니다 2023

## 주요업무

In [ ]:
df['주요업무'].nunique()

730

In [ ]:
preprocessing(df, '주요업무')

In [ ]:
# '주요업무' 라는 단어를 기준으로 나눈 뒤, 마지막 부분만 남기기
def extract_last_text(row):
    segments = row.split('주요업무')
    if len(segments) > 1:
        return segments[-1].strip()
    else:
        return row

df['주요업무'] = df['주요업무'].apply(extract_last_text)

In [ ]:
df['주요업무'].tolist()

['기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다\\n  LLM 학습 및 finetuning을 통해 내부적으로 필요한 LLM 개발을 진행합니다\\n  AI ,  ML 개발 방향성에 대한 전략을 수립합니다\\n  기업 고객에게 필요한 LLM 학습 데이터 구축 가이드라인 수립 및 사내 데이터 구축을 지원합니다\\n',
 '\\n MLIR 기반의 AI 컴파일러 관련 프로젝트에 참여합니다\\n',
 'LLM 모델 연구 및 고도화\\n\\nLLM 연구의 서비스화를 위한 개발 지원\\n',
 'sLLM 모델 연구개발 및 서비스 고도화 \\n\\nsLLM 기반의 서비스 모델 개발 및 튜닝 리딩\\n\\n개발 프로젝트 관리 및 협업 리딩\\n\\nLLM 관련 방법론 등 세션 리딩\\n\\nLLM 기술 논문화 또는 특허화 리딩\\n',
 'AI 개발\\n보철물 자동 디자인을 위한 AI 모델 개발\\n보철물 자동 디자인 서비스 운영\\n환자 데이터 관리\\n\\n Product Engineer\\n웹서비스 개발 FE,  BE\\n웹서비스 기획\\n\\n\\n채용절차\\n 서류 접수\\n 과제 전형  interview\\n     실무 역량을 확인하기 위해 기술 과제 전형이 진행됩니다\\n     경험,  인성 등 다양한 이야기를 공유하는 면접이 같이 진행됩니다\\n 처우 협의 및 합류\\n',
 '\\방대한 양의 인간 행동 데이터 수집하고,  마이닝하여 인사이트를 도출 하고,  의사결정을 하기 까지\\ 일련의 과정을 프레임워크로 만들고 제품화개발\\n 데이터 사이언스팀 소속으로 Tech 팀과 긴밀히 협력하여 세계적 수준의 A, B 실험 플랫폼,  고객 데이터 분석 플랫폼,  비지니스 의사결정 AI assistant 구축\\n 핵클 AI 개발에 필요한 아이디어와 전략 수립\\n',
 'RAGRetrieval Augmented Generation 시스템을 이용한 도메인 기반 챗봇 연구 및 개발 \\n 자체 LLMLarge Language Mode

## 자격요건

In [ ]:
preprocessing(df, '자격요건')

In [ ]:
df['자격요건'] = df['자격요건'].str.split('companyname').str[0]

In [ ]:
df['자격요건'].tolist()

['AI ,  ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분\\n  ML 프레임워크PyTorch,  TensorFlow 등를 자신있게 다룰 수 있는 분\\n',
 '\\n 웬만한 중견기업 못지 않은 복지 정책을 지원\\n 임직원 연차 구분 없이 20일 휴가\\n 임직원 연차 구분 없이 20일 워케이션 정직원\\n 학사, 석사, 박사 학비 지원 이사회 결의\\n 개인 상담 비용 1년 10회까지 비용의 80 지원 정직원\\n 재택이 어렵지만 출근도 어려울 경우 공유오피스 비용을 월 20만원까지 지원\\n 월 5만원의 사무용품, 간식비를 지원\\n 소프트웨어 공제조합과 과학기술인 공제회를 통한 여러 복지 서비스를 제공\\n 과학기술인 공제조합에서 퇴직금을 관리해 퇴직금 보호\\n 생명, 손해보험에 가입,  키보드 치다 손가락 다치면 보험 청구가 가능 정직원\\n\\n\\n 온라인으로 지원할 경우 자기소개 란에 TSN Lab에 지원하는 이유에 대해 5줄 이상매우 중요 적어주세요\\n 채용 절차는 서류 심사  사전 면접  기술 면접  인적성 검사 순으로 진행됩니다\\n\\n, ',
 '경력 3년 이상 또는 관련 학과 석사 학위 소지자\\n\\nPyTorch,  Tensorflow와 같은 딥러닝 프레임워크에 대한 지식 및 개발 경험\\n\\nAttention 메커니즘을 포함한 Transformers에 대한 이해도가 높으신 분\\n\\nLangChain,  AutoGPT,  LlamaIndex,  SimpleAIChat,  MetaGPT,  AuoChain,  Llama2Accessory,  PEFT,  QLora 등 LLM 관련 라이브러리 및 프레임워크에 대한 경험이 있으신 분\\n',
 'AI 관련학과 박사학위 소지자 또는 이에 준하는 경력 보유자\\n\\nLLM 관련 최신 논문 등의 기술 구현 가능 하신 분\\n',
 'python 또는 TypeScript 로 개발 경험 있으신분\\n Open mind 를 가지고 새로운 것을 두려워 하지 않고 다양한 역

## 우대사항

In [ ]:
preprocessing(df, '우대사항')

In [ ]:
df['우대사항'] = df['우대사항'].str.lstrip('[사항\\n] ').str.strip('[\\n] ')
df['우대사항'] = df['우대사항'].str.split('commonclose').str[0]

In [ ]:
df['우대사항'].tolist()

['모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이 있는 분\\n  자신이 만든 모델을 실제 상용 서비스에 적용하고 운영한 경험이 있는 분\\n  AWS나 Azure 등 클라우드 환경을 이용해 개발한 경험이 있는 분\\n  Github에 공개 가능한 ML 프로젝트가 있는 분',
 nan,
 '인공지능 관련 대회해커톤,  캐글,  그랜드챌린지 등의 출전 경험 및 입상\\n\\nMultimodal LLM에 대한 연구에 관심이 있으신 분\\n\\nAttention 메커니즘을 포함한 Transformers에 대한 이해도가 높으신 분\\n\\nFastAPI,  Ray,  Optuna,  Docker 등에 대한 활용 경험이 있으신 분\\n\\n연구 세션 진행 등 AI 교육에 관심이 있으신 분\\n\\n새로운 기술에 대한 관심이 많고,  자기개발을 위해 노력하시는 분',
 'AI 관련 Top tier 학회NeurIPS,  CVPR,  AAAI 등 논문 게재 실적이 있으신 분\\n\\nLangChain,  LlamaIndex,  HayStack 등의  연동 플랫폼 구조에 대한 이해가 있으신 분\\n\\nSpring Framework,  Java 기반 어플리케이션에 대한 이해가 있으신 분\\n\\n이해관계자 커뮤니케이션,  발표 등에 능숙 하신 분',
 '성장하는 스타트업에 관심 있으신 분',
 'chatGPT,  LLaMA 등의 API를 활용한 서비스 개발 경험 \\n Frontend는 익숙치 않아도 Backend에 강점이 있고 ML,  AI에 평소 호기심이 많은 분\\n\\n\\n Python,  Java,  Kotlin,  Kotlin Coroutines,  React,  Typescript\\n Spring FrameworkMVC,  WebFlux\\n JPA, Hibernate,  Mysql\\n Kafka,  Elasticsearch,  Redis\\n AWS',
 '이공계 석사 이상 또는 이에 준하는 2년 이상의 경력을 지니신 

## 혜택 및 복지

In [ ]:
preprocessing(df, '혜택 및 복지')

In [ ]:
df['혜택 및 복지'] = df['혜택 및 복지'].str.split('companyname').str[0]
df['혜택 및 복지'] = df['혜택 및 복지'].str.split('title').str[0]
df['혜택 및 복지'] = df['혜택 및 복지'].str.replace('  ',' ').str.strip()

In [ ]:
df['혜택 및 복지'].tolist()

['상호 존중하는 문화\\n  직급 없는 \\님\\ 호칭 및 경어 사용, 자율 복장 등\\n  자유로운 연차 사용과 사생활에 대한 배려와 존중\\n\\n 복리후생\\n  연봉 外 중식대 별도 제공 앱 활용, 10, 000원, 일\\n  경조휴가 및 경조지원금 ex 결혼 50만원, 출산 100만원 등\\n  기념일 휴가 ex 생일휴가, 결혼기념일 반차 등\\n  근속기간에 따른 추가 복리후생 \\n   명절 상여, 건강검진, 자기개발비, 사내대출 지원 등\\n\\n 개발장비\\n  맥북 4K 모니터 개발장비 지원비최초 입사시 15만원\\n\\n,',
 '\\n 웬만한 중견기업 못지 않은 복지 정책을 지원\\n 임직원 연차 구분 없이 20일 휴가\\n 임직원 연차 구분 없이 20일 워케이션 정직원\\n 학사, 석사, 박사 학비 지원 이사회 결의\\n 개인 상담 비용 1년 10회까지 비용의 80 지원 정직원\\n 재택이 어렵지만 출근도 어려울 경우 공유오피스 비용을 월 20만원까지 지원\\n 월 5만원의 사무용품, 간식비를 지원\\n 소프트웨어 공제조합과 과학기술인 공제회를 통한 여러 복지 서비스를 제공\\n 과학기술인 공제조합에서 퇴직금을 관리해 퇴직금 보호\\n 생명, 손해보험에 가입, 키보드 치다 손가락 다치면 보험 청구가 가능 정직원\\n\\n\\n 온라인으로 지원할 경우 자기소개 란에 TSN Lab에 지원하는 이유에 대해 5줄 이상매우 중요 적어주세요\\n 채용 절차는 서류 심사 사전 면접 기술 면접 인적성 검사 순으로 진행됩니다\\n\\n,',
 '유연 근무제\\n 자유 복장\\n 최신 장비\\n 수평적 조직\\n 도서 구입비 지원\\n 야간근무자 택시비 지원\\n\\n,',
 '유연 근무제\\n 자유 복장\\n 최신 장비\\n 수평적 조직\\n 도서 구입비 지원\\n 야간근무자 택시비 지원\\n\\n,',
 '경쟁력 있는 연봉\\n   협의를 통해 충분히 경쟁력 있는 수준의 연봉을 제시합니다\\n\\n 스톡옵션 부여\\n   초기 스타트업인 만큼 스톡옵

## 기술스택

In [ ]:
skillsets = Counter([s for ss in df['기술스택'].str.split(', ') for s in ss]).most_common()
skillsets

[('Python', 421),
 ('None', 365),
 ('AWS', 270),
 ('Git', 193),
 ('JavaScript', 155),
 ('React', 146),
 ('Java', 144),
 ('SQL', 135),
 ('Docker', 126),
 ('Pytorch', 125),
 ('MySQL', 124),
 ('TypeScript', 119),
 ('Linux', 106),
 ('Github', 104),
 ('Tensorflow', 95),
 ('Spring Framework', 84),
 ('Android', 79),
 ('C++', 78),
 ('Node.js', 76),
 ('Kotlin', 69),
 ('JIRA', 65),
 ('PostgreSQL', 64),
 ('HTML', 61),
 ('MongoDB', 58),
 ('Kubernetes', 54),
 ('iOS', 53),
 ('C', 50),
 ('Swift', 50),
 ('React.js', 50),
 ('CSS', 45),
 ('React Native', 42),
 ('C / C++', 40),
 ('Next.js', 40),
 ('Spark', 38),
 ('ML', 37),
 ('NodeJS', 37),
 ('C#', 34),
 ('Azure', 34),
 ('Spring Boot', 34),
 ('Hadoop', 31),
 ('Slack', 30),
 ('Notion', 30),
 ('R', 29),
 ('Tableau', 29),
 ('FastAPI', 28),
 ('GCP', 28),
 ('DevOps', 28),
 ('NoSQL', 27),
 ('ElasticSearch', 25),
 ('MSSQL', 25),
 ('Redis', 24),
 ('OpenCV', 23),
 ('Confluence', 23),
 ('Django', 23),
 ('RDBMS', 23),
 ('Flask', 22),
 ('JPA', 22),
 ('Nest.js', 22),

## 결측치 확인, 외국계기업 제거, 중복 값 제거, 컬럼 병합 및 삭제

In [ ]:
# 회사소개, 마감기한은 분석에 사용하지 않을 컬럼
# 우대사항 결측치 확인해보자

df.isnull().sum()

공고명          0
직무           0
업종           0
유사 직무        0
회사명          0
회사소개        27
주요업무         0
자격요건         0
우대사항        17
혜택 및 복지      0
마감기한       979
기술스택         0
dtype: int64

In [ ]:
# 찾아보니 크롤링 오류가 아니라, 정말 우대사항이 없는 채용공고임.

df[df.우대사항.isnull()].head()

,공고명,직무,업종,유사 직무,회사명,회사소개,주요업무,자격요건,우대사항,혜택 및 복지,마감기한,기술스택
1,AI 컴파일러 개발자 (1년 이상),머신러닝 엔지니어,"IT, 컨텐츠",머신러닝 엔지니어,티에스엔랩,"실시간 AI, 실시간 통신 전문 기업 TSN Lab에서 네트워크 AI 개발자를 모십...",\n MLIR 기반의 AI 컴파일러 관련 프로젝트에 참여합니다\n,\n 웬만한 중견기업 못지 않은 복지 정책을 지원\n 임직원 연차 구분 없이 20일...,NaN,[연봉은 어떻게 되나요?]\n• 경력자의 경우 기존의 경력과 동일한 분야인 경우 기...,NaN,None
15,AI 엔지니어,머신러닝 엔지니어,"IT, 컨텐츠","데이터 엔지니어,파이썬 개발자,머신러닝 엔지니어",프롬나드에이아이,프롬나드에이아이는 지식노동자들이 업무에 투여하는 시간의 질을 바꿔 놓을 서비스를 만...,LLM기반 모델 리서치 및 AI 기술 RD \n LLM유관 로직 구현 및 서비스 개...,컴퓨터 공학 석사 이상 AI전공 선호\n\n혜택 및 복지\n 최신장비와 소프트웨어 ...,NaN,• 최신장비와 소프트웨어 지원\n• 도서구입비 지원\n• 교육비/ 세미나비 등 지원...,NaN,None
17,Python application & Algorithm 개발자,머신러닝 엔지니어,"IT, 컨텐츠","파이썬 개발자,머신러닝 엔지니어,소프트웨어 엔지니어",메디픽셀,Medipixel is a medical imaging AI startup comp...,"혈관조영술 기반 혈관 분할, 병변의 정량적 분석 등의 모델 개발\n 혈관조영술 기...","Application 개발 및 유지 보수, 배포 프로세스 관리\n 의료 도메인 지...",NaN,• 자율 재택근무 및 자율출퇴근제 시행 (MOOC제도)\n• 식대 별도지원\n• 간...,NaN,None
125,[Data and Analytics] Senior Data Analyst,머신러닝 엔지니어,"IT, 컨텐츠","데이터 엔지니어,머신러닝 엔지니어",Electronic Arts Korea (EA Korea),Data and Analytics DnA팀은 EA SPORTS FC Online에서...,게임의 기획 및 비즈니스 모델 수립에 데이터 기반의 의사 결정을 위한 데이터 분석 ...,"공학, 컴퓨터 과학 또는 관련 분야에서 학사 또는 석사 학위 보유자\n 데이터 분...",NaN,• 연 17일의 연차휴가\n• 일주일간의 크리스마스 유급휴가 \n• 단체상해보험 및...,NaN,None
329,딥러닝 모델 개발자,머신러닝 엔지니어,"IT, 컨텐츠","데이터 사이언티스트,머신러닝 엔지니어,소프트웨어 엔지니어",보이저엑스(VoyagerX),보이저엑스는 더욱 즐겁고 편리한 삶을 돕는 인공지능 소프트웨어를 개발하는 스타트업 ...,"업무내용\n Vrew, vFlat 등 인공지능 서비스에 필요한 모델을 개발 세상에...",A 모든 개발자 공통 요건\n 개발을 좋아함\n 수학을 좋아함\n 딥러닝에 관심이 ...,NaN,• 휴가사용은 자유롭게 + (무급) 휴가 무제한\n• 동료와 원활한 협업을 위해 1...,NaN,None


In [ ]:
# 마감기한은 사용하지 않으므로 제거
df.drop('마감기한', axis=1, inplace=True)
df.head()

,공고명,직무,업종,유사 직무,회사명,회사소개,주요업무,자격요건,우대사항,혜택 및 복지,기술스택
0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"IT, 컨텐츠","데이터 엔지니어,머신러닝 엔지니어",플리토,플리토는 유수의 글로벌 IT 기업들과 함께 언어 장벽 없는 세계를 만들어 가고 있는...,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다\...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분\n ML ...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"- 상호 존중하는 문화\n : 직급 없는 \""님\"" 호칭 및 경어 사용, 자율 ...","Github, Android, Firebase, Kotlin, Coroutine"
1,AI 컴파일러 개발자 (1년 이상),머신러닝 엔지니어,"IT, 컨텐츠",머신러닝 엔지니어,티에스엔랩,"실시간 AI, 실시간 통신 전문 기업 TSN Lab에서 네트워크 AI 개발자를 모십...",\n MLIR 기반의 AI 컴파일러 관련 프로젝트에 참여합니다\n,\n 웬만한 중견기업 못지 않은 복지 정책을 지원\n 임직원 연차 구분 없이 20일...,NaN,[연봉은 어떻게 되나요?]\n• 경력자의 경우 기존의 경력과 동일한 분야인 경우 기...,None
2,LLM 연구원 (자연어 처리),머신러닝 엔지니어,"IT, 컨텐츠",머신러닝 엔지니어,딥노이드(Deepnoid),LLM 연구원 자연어 처리\n\n\n,LLM 모델 연구 및 고도화\n\nLLM 연구의 서비스화를 위한 개발 지원\n,"경력 3년 이상 또는 관련 학과 석사 학위 소지자\n\nPyTorch, Tenso...","인공지능 관련 대회해커톤, 캐글, 그랜드챌린지 등의 출전 경험 및 입상\n\nM...",1. 유연 근무제\n2. 자유 복장\n3. 최신 장비\n4. 수평적 조직\n5. 도...,"C#, C, C++, WPF"
3,AI R&D 팀장,머신러닝 엔지니어,"IT, 컨텐츠",머신러닝 엔지니어,딥노이드(Deepnoid),AI RD 팀장\n\n\n,sLLM 모델 연구개발 및 서비스 고도화 \n\nsLLM 기반의 서비스 모델 개발 ...,AI 관련학과 박사학위 소지자 또는 이에 준하는 경력 보유자\n\nLLM 관련 최신...,"AI 관련 Top tier 학회NeurIPS, CVPR, AAAI 등 논문 게재...",1. 유연 근무제\n2. 자유 복장\n3. 최신 장비\n4. 수평적 조직\n5. 도...,"C#, C, C++, WPF"
4,소프트웨어 엔지니어,머신러닝 엔지니어,"IT, 컨텐츠","웹 개발자,머신러닝 엔지니어,소프트웨어 엔지니어","스타인펠드(Steinfeld,Inc.)",Steinfeldco는 치과 보철치료에 필요한 보철물을 더 효율적으로 제공하기 위해...,AI 개발\n보철물 자동 디자인을 위한 AI 모델 개발\n보철물 자동 디자인 서비스...,python 또는 TypeScript 로 개발 경험 있으신분\n Open mind ...,성장하는 스타트업에 관심 있으신 분,• 경쟁력 있는 연봉\n - 협의를 통해 충분히 경쟁력 있는 수준의 연봉을 제...,"Git, Firebase, Google Cloud Platform, Pytorch,..."


In [ ]:
df['기술스택'].head()

0         Github, Android, Firebase, Kotlin, Coroutine
1                                                 None
2                                      C#, C, C++, WPF
3                                      C#, C, C++, WPF
4    Git, Firebase, Google Cloud Platform, Pytorch,...
Name: 기술스택, dtype: object

In [ ]:
df['기술스택'] = df['기술스택'].str.replace('None', '')

# 이 중에 랜덤으로 하나 뽑아서 기술 스택 뒤에 붙이기

conditions = ['사용이 가능하신 분', '사용 경험이 있으신 분', '사용 가능자',
              '활용이 가능하신 분', '활용 경험이 있으신 분', '활용 가능자',
              '에 대한 이해도가 있으신 분', '에 대한 이해', '경험이 있으신 분', '다룰 수 있으신 분']

# 기술 스택 업데이트 함수
def update_skill_description(skill):
    if skill and skill.strip():  # 공백이 아닌 경우에만 적용
        condition = random.choice(conditions)
        return f"{skill.strip()} {condition}"
    else:
        return skill

# 기술 스택 업데이트 적용
df['기술스택_long'] = df['기술스택'].apply(lambda x: ', '.join([update_skill_description(skill) for skill in (x.split(',') if x else [])]))

# 쉼표 -> 줄바꿈 문자
df['기술스택'] = df['기술스택'].str.replace(',', r'\n')
df['기술스택_long'] = df['기술스택_long'].str.replace(',', r'\n')

df['기술스택_long'].head()

0    Github 에 대한 이해도가 있으신 분\n Android 사용 경험이 있으신 분\...
1                                                     
2    C# 사용 가능자\n C 사용 가능자\n C++ 활용 경험이 있으신 분\n WPF ...
3    C# 다룰 수 있으신 분\n C 활용 경험이 있으신 분\n C++ 사용 가능자\n ...
4    Git 경험이 있으신 분\n Firebase 에 대한 이해도가 있으신 분\n Goo...
Name: 기술스택_long, dtype: object

In [ ]:
# 주요업무, 자격요건, 우대사항 컬럼에서 한글이 등장하지 않으면(외국계 기업) 값을 공백으로 바꿈
df['주요업무'] = df['주요업무'].astype(str).apply(lambda x: '' if not pd.Series(x).str.contains('[가-힣]').any() else x)
df['자격요건'] = df['자격요건'].astype(str).apply(lambda x: '' if not pd.Series(x).str.contains('[가-힣]').any() else x)
df['우대사항'] = df['우대사항'].astype(str).apply(lambda x: '' if not pd.Series(x).str.contains('[가-힣]').any() else x)

In [ ]:
# 중복값 제거
df = df.copy()
df = df.drop_duplicates()

In [ ]:
# 주요업무, 자격요건, 우대사항, (기술스택_long) 열을 결합

# combined_token : 문장 토큰화에 사용
# combined : 형태소 분석에 사용

df['combined_token'] = df['주요업무'].fillna('') + r'\n' + df['자격요건'].fillna('') + r'\n' + df['우대사항'].fillna('') #+ r'\n' + df['기술스택_long'].fillna('')
df['combined'] = df['combined_token'].str.replace(r'\\n|\\t|\\b|\\|,', '', regex=True)
df.head()

,공고명,직무,업종,유사 직무,회사명,회사소개,주요업무,자격요건,우대사항,혜택 및 복지,기술스택,기술스택_long,combined_token,combined
0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"IT, 컨텐츠","데이터 엔지니어,머신러닝 엔지니어",플리토,플리토는 유수의 글로벌 IT 기업들과 함께 언어 장벽 없는 세계를 만들어 가고 있는...,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다\...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분\n ML ...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"- 상호 존중하는 문화\n : 직급 없는 \""님\"" 호칭 및 경어 사용, 자율 ...",Github\n Android\n Firebase\n Kotlin\n Coroutine,Github 에 대한 이해도가 있으신 분\n Android 사용 경험이 있으신 분\...,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다\...,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다 ...
1,AI 컴파일러 개발자 (1년 이상),머신러닝 엔지니어,"IT, 컨텐츠",머신러닝 엔지니어,티에스엔랩,"실시간 AI, 실시간 통신 전문 기업 TSN Lab에서 네트워크 AI 개발자를 모십...",\n MLIR 기반의 AI 컴파일러 관련 프로젝트에 참여합니다\n,\n 웬만한 중견기업 못지 않은 복지 정책을 지원\n 임직원 연차 구분 없이 20일...,,[연봉은 어떻게 되나요?]\n• 경력자의 경우 기존의 경력과 동일한 분야인 경우 기...,,,\n MLIR 기반의 AI 컴파일러 관련 프로젝트에 참여합니다\n\n\n 웬만한 중...,MLIR 기반의 AI 컴파일러 관련 프로젝트에 참여합니다 웬만한 중견기업 못지 않...
2,LLM 연구원 (자연어 처리),머신러닝 엔지니어,"IT, 컨텐츠",머신러닝 엔지니어,딥노이드(Deepnoid),LLM 연구원 자연어 처리\n\n\n,LLM 모델 연구 및 고도화\n\nLLM 연구의 서비스화를 위한 개발 지원\n,"경력 3년 이상 또는 관련 학과 석사 학위 소지자\n\nPyTorch, Tenso...","인공지능 관련 대회해커톤, 캐글, 그랜드챌린지 등의 출전 경험 및 입상\n\nM...",1. 유연 근무제\n2. 자유 복장\n3. 최신 장비\n4. 수평적 조직\n5. 도...,C#\n C\n C++\n WPF,C# 사용 가능자\n C 사용 가능자\n C++ 활용 경험이 있으신 분\n WPF ...,LLM 모델 연구 및 고도화\n\nLLM 연구의 서비스화를 위한 개발 지원\n\n경...,LLM 모델 연구 및 고도화LLM 연구의 서비스화를 위한 개발 지원경력 3년 이상 ...
3,AI R&D 팀장,머신러닝 엔지니어,"IT, 컨텐츠",머신러닝 엔지니어,딥노이드(Deepnoid),AI RD 팀장\n\n\n,sLLM 모델 연구개발 및 서비스 고도화 \n\nsLLM 기반의 서비스 모델 개발 ...,AI 관련학과 박사학위 소지자 또는 이에 준하는 경력 보유자\n\nLLM 관련 최신...,"AI 관련 Top tier 학회NeurIPS, CVPR, AAAI 등 논문 게재...",1. 유연 근무제\n2. 자유 복장\n3. 최신 장비\n4. 수평적 조직\n5. 도...,C#\n C\n C++\n WPF,C# 다룰 수 있으신 분\n C 활용 경험이 있으신 분\n C++ 사용 가능자\n ...,sLLM 모델 연구개발 및 서비스 고도화 \n\nsLLM 기반의 서비스 모델 개발 ...,sLLM 모델 연구개발 및 서비스 고도화 sLLM 기반의 서비스 모델 개발 및 튜닝...
4,소프트웨어 엔지니어,머신러닝 엔지니어,"IT, 컨텐츠","웹 개발자,머신러닝 엔지니어,소프트웨어 엔지니어","스타인펠드(Steinfeld,Inc.)",Steinfeldco는 치과 보철치료에 필요한 보철물을 더 효율적으로 제공하기 위해...,AI 개발\n보철물 자동 디자인을 위한 AI 모델 개발\n보철물 자동 디자인 서비스...,python 또는 TypeScript 로 개발 경험 있으신분\n Open mind ...,성장하는 스타트업에 관심 있으신 분,• 경쟁력 있는 연봉\n - 협의를 통해 충분히 경쟁력 있는 수준의 연봉을 제...,Git\n Firebase\n Google Cloud Platform\n Pytor...,Git 경험이 있으신 분\n Firebase 에 대한 이해도가 있으신 분\n Goo...,AI 개발\n보철물 자동 디자인을 위한 AI 모델 개발\n보철물 자동 디자인 서비스...,AI 개발보철물 자동 디자인을 위한 AI 모델 개발보철물 자동 디자인 서비스 운영환...


## 결과 확인

In [ ]:
# 컬럼별 문자열 다시 길이 확인 -> 이젠 좀 정상

num_df = pd.DataFrame()

for col in ['공고명', '직무', '업종', '유사 직무', '회사명', '회사소개', '주요업무', '자격요건', '우대사항', '혜택 및 복지']:
    num_df[f'{col}_num'] = (df[col].dropna().map(lambda x: len(x)))

num_df.describe()

,공고명_num,직무_num,업종_num,유사 직무_num,회사명_num,회사소개_num,주요업무_num,자격요건_num,우대사항_num,혜택 및 복지_num
count,1145.000000,1145.000000,1145.000000,1145.000000,1145.000000,1118.000000,1145.000000,1145.000000,1145.000000,1145.000000
mean,20.264629,8.447162,6.606114,21.806114,6.966812,542.435599,210.631441,187.767686,181.247162,439.409607
std,10.519081,1.639507,1.473352,8.358602,4.975353,399.999372,240.898933,135.759416,159.892236,409.664609
min,3.000000,3.000000,2.000000,0.000000,2.000000,14.000000,0.000000,0.000000,0.000000,1.000000
25%,12.000000,8.000000,7.000000,17.000000,4.000000,281.000000,86.000000,106.000000,87.000000,175.000000
50%,18.000000,9.000000,7.000000,25.000000,6.000000,462.000000,146.000000,163.000000,144.000000,318.000000
75%,27.000000,9.000000,7.000000,29.000000,9.000000,730.000000,246.000000,234.000000,224.000000,576.000000
max,56.000000,10.000000,11.000000,39.000000,46.000000,4397.000000,2315.000000,1164.000000,1844.000000,3101.000000


In [ ]:
df.head()

,공고명,직무,업종,유사 직무,회사명,회사소개,주요업무,자격요건,우대사항,혜택 및 복지,기술스택,기술스택_long,combined_token,combined
0,"AI, ML Engineer 박사급",머신러닝 엔지니어,"IT, 컨텐츠","데이터 엔지니어,머신러닝 엔지니어",플리토,플리토는 유수의 글로벌 IT 기업들과 함께 언어 장벽 없는 세계를 만들어 가고 있는...,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다\...,"AI , ML 등 관련 분야에서 박사 학위를 받았거나 받을 예정인 분\n ML ...",모델을 직접 만들었거나 공개된 모델을 finetuning해서 성능을 향상시킨 경험이...,"상호 존중하는 문화\n 직급 없는 \님\ 호칭 및 경어 사용, 자율 복장 등\n ...",Github\n Android\n Firebase\n Kotlin\n Coroutine,Github 활용 가능자\n Android 사용이 가능하신 분\n Firebase ...,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다\...,기존 플리토 번역기에 신규 언어쌍 추가 및 플리토 번역기 성능 고도화를 담당합니다 ...
1,AI 컴파일러 개발자 (1년 이상),머신러닝 엔지니어,"IT, 컨텐츠",머신러닝 엔지니어,티에스엔랩,"실시간 AI, 실시간 통신 전문 기업 TSN Lab에서 네트워크 AI 개발자를 모십...",\n MLIR 기반의 AI 컴파일러 관련 프로젝트에 참여합니다\n,\n 웬만한 중견기업 못지 않은 복지 정책을 지원\n 임직원 연차 구분 없이 20일...,,\n 웬만한 중견기업 못지 않은 복지 정책을 지원\n 임직원 연차 구분 없이 20일...,,,\n MLIR 기반의 AI 컴파일러 관련 프로젝트에 참여합니다\n\n\n 웬만한 중...,MLIR 기반의 AI 컴파일러 관련 프로젝트에 참여합니다 웬만한 중견기업 못지 않...
2,LLM 연구원 (자연어 처리),머신러닝 엔지니어,"IT, 컨텐츠",머신러닝 엔지니어,딥노이드(Deepnoid),LLM 연구원 자연어 처리\n\n\n,LLM 모델 연구 및 고도화\n\nLLM 연구의 서비스화를 위한 개발 지원\n,"경력 3년 이상 또는 관련 학과 석사 학위 소지자\n\nPyTorch, Tenso...","인공지능 관련 대회해커톤, 캐글, 그랜드챌린지 등의 출전 경험 및 입상\n\nM...",유연 근무제\n 자유 복장\n 최신 장비\n 수평적 조직\n 도서 구입비 지원\n ...,C#\n C\n C++\n WPF,C# 사용 경험이 있으신 분\n C 활용 경험이 있으신 분\n C++ 사용 경험이 ...,LLM 모델 연구 및 고도화\n\nLLM 연구의 서비스화를 위한 개발 지원\n\n경...,LLM 모델 연구 및 고도화LLM 연구의 서비스화를 위한 개발 지원경력 3년 이상 ...
3,AI R&D 팀장,머신러닝 엔지니어,"IT, 컨텐츠",머신러닝 엔지니어,딥노이드(Deepnoid),AI RD 팀장\n\n\n,sLLM 모델 연구개발 및 서비스 고도화 \n\nsLLM 기반의 서비스 모델 개발 ...,AI 관련학과 박사학위 소지자 또는 이에 준하는 경력 보유자\n\nLLM 관련 최신...,"AI 관련 Top tier 학회NeurIPS, CVPR, AAAI 등 논문 게재...",유연 근무제\n 자유 복장\n 최신 장비\n 수평적 조직\n 도서 구입비 지원\n ...,C#\n C\n C++\n WPF,C# 다룰 수 있으신 분\n C 사용 경험이 있으신 분\n C++ 다룰 수 있으신 ...,sLLM 모델 연구개발 및 서비스 고도화 \n\nsLLM 기반의 서비스 모델 개발 ...,sLLM 모델 연구개발 및 서비스 고도화 sLLM 기반의 서비스 모델 개발 및 튜닝...
4,소프트웨어 엔지니어,머신러닝 엔지니어,"IT, 컨텐츠","웹 개발자,머신러닝 엔지니어,소프트웨어 엔지니어","스타인펠드(Steinfeld,Inc.)",Steinfeldco는 치과 보철치료에 필요한 보철물을 더 효율적으로 제공하기 위해...,AI 개발\n보철물 자동 디자인을 위한 AI 모델 개발\n보철물 자동 디자인 서비스...,python 또는 TypeScript 로 개발 경험 있으신분\n Open mind ...,성장하는 스타트업에 관심 있으신 분,경쟁력 있는 연봉\n 협의를 통해 충분히 경쟁력 있는 수준의 연봉을 제시합니다\...,Git\n Firebase\n Google Cloud Platform\n Pytor...,Git 사용이 가능하신 분\n Firebase 활용 경험이 있으신 분\n Googl...,AI 개발\n보철물 자동 디자인을 위한 AI 모델 개발\n보철물 자동 디자인 서비스...,AI 개발보철물 자동 디자인을 위한 AI 모델 개발보철물 자동 디자인 서비스 운영환...


In [ ]:
df.isnull().sum()

공고명                0
직무                 0
업종                 0
유사 직무              0
회사명                0
회사소개              27
주요업무               0
자격요건               0
우대사항               0
혜택 및 복지            0
기술스택               0
기술스택_long          0
combined_token     0
combined           0
dtype: int64

In [ ]:
df.to_csv('/content/drive/MyDrive/응용통계학과 공모전/data/wanted_real_final_preprocessed.csv', index= False)